# Helpful utility functions

This notebook demonstrates how to use various utility functions related to interacting with CoLink storage. 

Every served flow has associated entries in CoLink storage. The base path to these entries corresponds to the flow_endpoint argument passed when serving the flow. The typical structure includes:
```
endpoint/
    init = 0/1
    singleton = 0/1
    parallel_dispatch = 0/1
    flow_class_name = ...
    default_dispatch_point = ...
    mounts/
       id_of_user_who_mounted_the_flow/
           flow_instance_id/
               init = 0/1
               config_overrides = ...
```

Calling serve_flow will create the initial entries under an endpoint and calling get_flow_instance will create an entry under the mounts/ path. If a Flow was served as a singleton, there will only be at most one entry under the mounts/ path.

You can see the file structure visually by connecting [this frontend](https://colink.run/) to your CoLink server.

Util functions include:
- serving.is_flow_served
- serving.unserve_flow
- serving.delete_flow_instance
- serving.delete_flow_endpoint
- serving.delete_all_flow_endpoints
- colink_utils.print_flow_instances
- colink_utils.print_served_flows
- serving._get_local_flow_instance_metadata

In [1]:
%load_ext autoreload
%autoreload 2
import os, json
from colink import CoLink
from aiflows.utils import serving
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread

/Users/nicolasbaldwin/opt/miniconda3/envs/mockenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cl = colink_utils.start_colink_server()

In [3]:
run_dispatch_worker_thread(cl)
run_dispatch_worker_thread(cl)

[2024-04-08 10:31:22,664][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-04-08 10:31:22,667][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:31:22,677][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-04-08 10:31:22,679][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch


In [4]:
serving.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic"
)
serving.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential"
)

[2024-04-08 10:31:24,370][aiflows.utils.serving:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
[2024-04-08 10:31:24,373][aiflows.utils.serving:117][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:31:24,435][aiflows.utils.serving:118][INFO] - parallel_dispatch: False
[2024-04-08 10:31:24,436][aiflows.utils.serving:119][INFO] - singleton: False

[2024-04-08 10:31:24,497][aiflows.utils.serving:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
[2024-04-08 10:31:24,499][aiflows.utils.serving:117][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:31:24,500][aiflows.utils.serving:118][INFO] - parallel_dispatch: False
[2024-04-08 10:31:24,503][aiflows.utils.serving:119][INFO] - singleton: False



True

In [5]:
colink_utils.print_served_flows(cl, print_values = True)

 reverse_number_atomic/
   init: 1
   flow_class_name: ReverseNumberFlowModule.ReverseNumberAtomicFlow
   singleton: False
   default_dispatch_point: b'coflows_dispatch'


   parallel_dispatch: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
 reverse_number_sequential/
   singleton: False
   init: 1
   parallel_dispatch: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
   default_dispatch_point: b'coflows_dispatch'
   flow_class_name: ReverseNumberFlowModule.ReverseNumberSequentialFlow


In [6]:
flow = serving.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
)
# we mount 3 instances: one ReverseNumberSequentialFlow and two ReverseNumberAtomicFlow

[2024-04-08 10:31:31,519][aiflows.utils.serving:336][INFO] - Mounted be54606d-5bb6-4885-aa5d-2fa8ac813186 at flows:reverse_number_atomic:mounts:local:be54606d-5bb6-4885-aa5d-2fa8ac813186
[2024-04-08 10:31:31,585][aiflows.utils.serving:336][INFO] - Mounted a3385620-a4f3-4cee-9492-ec83e4969cec at flows:reverse_number_atomic:mounts:local:a3385620-a4f3-4cee-9492-ec83e4969cec
[2024-04-08 10:31:31,634][aiflows.utils.serving:336][INFO] - Mounted b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5 at flows:reverse_number_sequential:mounts:local:b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5


In [7]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)

 reverse_number_atomic/
   init
   flow_class_name
   singleton


   mounts/
     local/
       be54606d-5bb6-4885-aa5d-2fa8ac813186/
         init
         config_overrides
       a3385620-a4f3-4cee-9492-ec83e4969cec/
         config_overrides
         init
   default_dispatch_point
   parallel_dispatch
 reverse_number_sequential/
   singleton
   mounts/
     local/
       b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5/
         config_overrides
         init
   init
   parallel_dispatch
   default_dispatch_point
   flow_class_name

Flow Instances:
 be54606d-5bb6-4885-aa5d-2fa8ac813186
 a3385620-a4f3-4cee-9492-ec83e4969cec
 b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5


In [8]:
serving._get_local_flow_instance_metadata(cl, flow_id=flow.get_instance_id())

{'flow_endpoint': 'reverse_number_sequential', 'user_id': 'local'}

In [9]:
serving.unserve_flow(cl, flow_endpoint="reverse_number_atomic")
# just sets init entry to 0, but all entries remain in storage
# flow instances of unserved endpoint remain alive

[2024-04-08 10:31:36,711][aiflows.utils.serving:199][INFO] - Stopped serving at flows:reverse_number_atomic


In [10]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)

 reverse_number_atomic/
   flow_class_name
   singleton
  

 mounts/
     local/
       be54606d-5bb6-4885-aa5d-2fa8ac813186/
         init
         config_overrides
       a3385620-a4f3-4cee-9492-ec83e4969cec/
         config_overrides
         init
   init
   default_dispatch_point
   parallel_dispatch
 reverse_number_sequential/
   singleton
   mounts/
     local/
       b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5/
         config_overrides
         init
   init
   parallel_dispatch
   default_dispatch_point
   flow_class_name

Flow Instances:
 be54606d-5bb6-4885-aa5d-2fa8ac813186
 a3385620-a4f3-4cee-9492-ec83e4969cec
 b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5


In [11]:
serving.is_flow_served(cl, flow_endpoint="reverse_number_atomic")

False

In [12]:
flow = serving.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
) # throws exception because a subflow is not served

FlowInstanceException: Failed to get flow instance at reverse_number_sequential served by user 03145b2e07327412873e25165a8dec78f82f4a97a7cd70396d058052bf733fec00.
Message: Failed to get instance of subflow first_reverse_flow.
Failed to get flow instance at reverse_number_atomic served by user 03145b2e07327412873e25165a8dec78f82f4a97a7cd70396d058052bf733fec00.
Message: Not serving at reverse_number_atomic.

In [13]:
serving.delete_flow_endpoint(cl, flow_endpoint="reverse_number_atomic")
# actually deletes entries at endpoint, including all flow instances mounted on the endpoint

[2024-04-08 10:31:57,881][aiflows.utils.serving:149][INFO] - Deleted flow instance be54606d-5bb6-4885-aa5d-2fa8ac813186
[2024-04-08 10:31:57,886][aiflows.utils.serving:149][INFO] - Deleted flow instance a3385620-a4f3-4cee-9492-ec83e4969cec


[2024-04-08 10:31:58,265][aiflows.utils.serving:158][INFO] - Stopped serving at flows:reverse_number_atomic


In [14]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)
# note that paths under reverse_number_atomic/ are empty (even though they get printed)

 reverse_number_atomic/
   mounts/
     local/
       be54606d-5bb6-4885-aa5d-2fa8ac813186/
       a3385620-a4f3-4cee-9492-ec83e4969cec/
 reverse_number_sequential/
   singleton
   mounts/
     local/
       b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5/
         config_overrides
         init
   init
   parallel_dispatch
   default_dispatch_point
   flow_class_name

Flow Instances:
 b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5


In [15]:
serving.delete_flow_instance(cl, flow_id=flow.get_instance_id())
# delets a single flow instance

[2024-04-08 10:32:04,012][aiflows.utils.serving:242][INFO] - Deleted flow instance b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5.


In [16]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)

 reverse_number_atomic/
   mounts/
     local/
       be54606d-5bb6-4885-aa5d-2fa8ac813186/
       a3385620-a4f3-4cee-9492-ec83e4969cec/
 reverse_number_sequential/
   singleton


   mounts/
     local/
       b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5/
   init
   parallel_dispatch
   default_dispatch_point
   flow_class_name

Flow Instances:


In [17]:
serving.delete_all_flow_endpoints(cl)
# deletes all CoLink entries associated with Flows

ERROR:root:DeleteEntry Received RPC exception: code=StatusCode.INTERNAL message=Key name not found: instance_metadata:be54606d-5bb6-4885-aa5d-2fa8ac813186


[2024-04-08 10:32:06,996][aiflows.utils.serving:152][INFO] - WARNING: flow be54606d-5bb6-4885-aa5d-2fa8ac813186 is mounted but it's metadata doesn't exist.


ERROR:root:DeleteEntry Received RPC exception: code=StatusCode.INTERNAL message=Key name not found: instance_metadata:a3385620-a4f3-4cee-9492-ec83e4969cec


[2024-04-08 10:32:07,017][aiflows.utils.serving:152][INFO] - WARNING: flow a3385620-a4f3-4cee-9492-ec83e4969cec is mounted but it's metadata doesn't exist.
[2024-04-08 10:32:07,063][aiflows.utils.serving:158][INFO] - Stopped serving at flows:reverse_number_atomic


ERROR:root:DeleteEntry Received RPC exception: code=StatusCode.INTERNAL message=Key name not found: instance_metadata:b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5


[2024-04-08 10:32:07,084][aiflows.utils.serving:152][INFO] - WARNING: flow b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5 is mounted but it's metadata doesn't exist.
[2024-04-08 10:32:07,264][aiflows.utils.serving:158][INFO] - Stopped serving at flows:reverse_number_sequential


In [18]:
colink_utils.print_served_flows(cl)
print("\nFlow Instances:")
colink_utils.print_flow_instances(cl)
# note that paths are empty (even though they get printed)

 reverse_number_atomic/
   mounts/
     local/
       be54606d-5bb6-4885-aa5d-2fa8ac813186/
       a3385620-a4f3-4cee-9492-ec83e4969cec/
 reverse_number_sequential/
   mounts/
     local/
       b9b131ea-a5d9-46e6-b1af-4e1b2f30fee5/

Flow Instances:
